## 유방암 데이터로 실습하기

데이터셋 정보
Breast Cancer Wisconsin Dataset

샘플 수: 569개 (악성 212개, 양성 357개)

특성 수: 30개 (종양의 크기, 모양, 질감 등)

태스크: 종양이 악성인지 양성인지 분류

#### 과제 2: 하이퍼파라미터 튜닝
GridSearchCV vs RandomSearchCV 비교

LogisticRegression의 최적 파라미터 찾기

정확도와 AUC 모두 확인

시간 효율성 비교

In [7]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, make_scorer # AUC를 기준으로 최적화
from scipy.stats import uniform
import time

# 데이터 불러오기 및 분리
data = load_breast_cancer()
X, y = data.data, data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
# 공통 모델 정의: LogisticRegression을 사용하며, solver는 'liblinear'로 고정하여 L1/L2 정규화를 모두 지원
#lr_model = LogisticRegression(solver='liblinear', random_state=42, max_iter=1000)
lr_model = LogisticRegression(random_state=42, max_iter=1000)

# 평가 기준 설정: AUC(Area Under the Curve)를 사용하여 모델의 분류 성능을 최적화
scoring = make_scorer(roc_auc_score) 


In [9]:
# A. Grid Search 탐색 공간 (이산적, 모든 조합)
# 6가지 C 값 * 2가지 Penalty = 총 12가지 조합
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100], 
    'penalty': ['l1', 'l2'] ,
    'solver': ['liblinear', 'saga']
} 

In [10]:
print("--- Grid Search 실행 중... ---")
start_time_grid = time.time()
grid_search = GridSearchCV(
    estimator=lr_model,
    param_grid=param_grid,
    cv=5, # 5-Fold Cross Validation
    scoring='accuracy',
    n_jobs=-1, # 병렬 처리
    verbose=2
)
grid_search.fit(X_train, y_train)
end_time_grid = time.time()

grid_time = end_time_grid - start_time_grid

--- Grid Search 실행 중... ---
Fitting 5 folds for each of 24 candidates, totalling 120 fits


In [11]:
grid_search.best_params_, grid_search.best_score_

({'C': 100, 'penalty': 'l1', 'solver': 'liblinear'},
 np.float64(0.9670329670329672))

In [12]:
best_model = grid_search.best_estimator_

In [13]:
best_model.predict(X_test)

array([1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1,
       0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1,
       0, 1, 0, 0])

In [14]:
best_model.score(X_test, y_test)

0.9824561403508771

In [15]:
# B. Random Search 탐색 공간 (연속적, 분포)
# C 값을 0.001에서 100.001 사이의 연속적인 균일 분포에서 무작위 샘플링
param_distributions = { 
    'C': uniform(loc=0.001, scale=100), 
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear', 'saga']
}

In [23]:
print("--- Randomized Search 실행 중... ---")
start_time_rand = time.time()
random_search = RandomizedSearchCV(
    estimator=lr_model,
    param_distributions=param_distributions,
    n_iter=12, # Grid Search와 동일하게 12회 샘플링
    cv=5, 
    scoring='accuracy',
    n_jobs=-1,
    random_state=42,
    verbose=2
)
random_search.fit(X_train, y_train)
end_time_rand = time.time()

rand_time = end_time_rand - start_time_rand

--- Randomized Search 실행 중... ---
Fitting 5 folds for each of 12 candidates, totalling 60 fits


In [24]:
random_search.best_params_, random_search.best_score_

({'C': np.float64(73.20039418114051), 'penalty': 'l1', 'solver': 'liblinear'},
 np.float64(0.9692307692307693))

In [25]:
random_best_model = random_search.best_estimator_

In [26]:
random_best_model.score(X_test, y_test)

0.9824561403508771

In [35]:
print("\n" + "="*50)
print("          하이퍼파라미터 튜닝 결과 비교")
print("="*50)

# Grid Search 결과
print("## 1. GridSearchCV 결과 (12 조합 완벽 탐색)")
print(f"   - 소요 시간: {grid_time:.4f} 초")
print(f"   - 최적 파라미터: C={grid_search.best_params_['C']:.4f}, penalty='{grid_search.best_params_['penalty']}'")
print(f"   - 최적 AUC (CV 평균): {grid_search.best_score_:.4f}")

# Randomized Search 결과
print("\n## 2. RandomizedSearchCV 결과 (12 조합 무작위 샘플링)")
print(f"   - 소요 시간: {rand_time:.4f} 초")
print(f"   - 최적 파라미터: C={random_search.best_params_['C']:.4f}, penalty='{random_search.best_params_['penalty']}'")
print(f"   - 최적 AUC (CV 평균): {random_search.best_score_:.4f}")

print("\n" + "-"*50)
print("## 시간 효율성")
print(f"시간 비교 (Random / Grid): {rand_time / grid_time * 100:.2f} % \n")




          하이퍼파라미터 튜닝 결과 비교
## 1. GridSearchCV 결과 (12 조합 완벽 탐색)
   - 소요 시간: 3.8263 초
   - 최적 파라미터: C=100.0000, penalty='l1'
   - 최적 AUC (CV 평균): 0.9670

## 2. RandomizedSearchCV 결과 (12 조합 무작위 샘플링)
   - 소요 시간: 0.6341 초
   - 최적 파라미터: C=73.2004, penalty='l1'
   - 최적 AUC (CV 평균): 0.9692

--------------------------------------------------
## 시간 효율성
시간 비교 (Random / Grid): 16.57 % 



randomizedSearchCV는 GridSearchCV와 동일한 12번의 시도를 하지만 소요시간이 3.8->0.6으로 약 83%단축됬다.
탐색 공간이 넓다면 무작위로 파라미터를 샘플링하는 RandomizedSearchCV가 훨씬 적은 비용으로 더 나은 성능을 발휘할 수 있다.

두 방법 모두 penaltysms l1을 최적의 정규화 방법으로 선택했다.
l1은 불필욯나 특성의 가중치를 0으로 만들기 때문에 유방암 데이터셋처럼 특성수가 많은 경우 유리하다. 